In [ ]:
'These Functions are Located in NetworkVisualizationCode and should be run in that file'

def get_path_to_substation(feeder, node):
    #returns list of edges between node and substation
    #node = node name as string
    graph = feeder.network
    node_path = []
    current_node = node
    
    for i in range(depths[node]):
        pred_node = list(graph.predecessors(current_node))
        node_path += [(current_node, pred_node[0])]
        current_node = pred_node[0]
    return node_path


def createRXmatrices(feeder, node_index_map):
    #calculated using phase A impedances unless there is no phase A then phase B else phase C
    graph = feeder.network
    n = len(graph.nodes) #number of nodes
    R = np.zeros((n,n))
    X = np.zeros((n,n))
    P = {} #initializing line path dictionary
   
    for node in graph.nodes:
        P[node] = get_path_to_substation(feeder, node)
    
    for n_outer in graph.nodes:
        index_outer = node_index_map[n_outer]
        
        for n_inner in graph.nodes:
            index_inner = node_index_map[n_inner]
            intersection_set = set.intersection(set(P[n_outer]), set(P[n_inner]))
            intersection_list = list(intersection_set)
            r_sum = 0
            x_sum = 0
            
            for edge in intersection_list:
                tot_edge_impedance = get_total_impedance_between_two_buses(feeder, edge[0], edge[1])
                
                if tot_edge_impedance['Phase 1']:
                    phase_edge_impedance = tot_edge_impedance['Phase 1']
                elif tot_edge_impedance['Phase 2']:
                    phase_edge_impedance = tot_edge_impedance['Phase 2']
                else:
                    phase_edge_impedance = tot_edge_impedance['Phase 3']  
                    
                r_sum += phase_edge_impedance.real
                x_sum += phase_edge_impedance.imag
            R[index_outer][index_inner] = 2*r_sum
            X[index_outer][index_inner] = 2*x_sum
            
    return R, X